### Import all necessary packages

In [232]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup
import requests
from lxml.html import parse

from urllib.request import urlopen
from pandas.io.parsers import TextParser
print('Libraries imported.')

Libraries imported.


## Extract data from web

In [256]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
doc = parse(urlopen(wikipedia_link))
tables = doc.xpath('//table')

def _unpack(row, kind='td'):
    elts = row.xpath('.//%s' %kind)
    return [val.text_content() for val in elts]




def parse_options_data(table):
    rows = table.xpath('.//tr')
    header = _unpack(rows[0], kind='th')
    data = [_unpack(r) for r in rows[1:]]
    return TextParser(data, names=header).get_chunk()

content = parse_options_data(tables[0])
content.index = range(len(content))

## Remove undefined Borough rows

In [257]:
# create content2
content2 = content.copy()
content2 =content2.drop(content2.index, inplace=False)

# filter not assigned Borough
index = 0
content2.columns = ['Postcode', 'Borough', 'Neighbourhood']
for i in content['Borough']:
    if i != "Not assigned":
#         print(content.loc[index])
        st = content["Neighbourhood\n"][index]
        pos=st.rfind('\n')
        st=st[:pos]
#         print("st: "+st)
        series = pd.Series({"Postcode":content["Postcode"][index],"Borough":content["Borough"][index],"Neighbourhood":st})
        content2 = content2.append(series,ignore_index=True)
    index += 1
    
#refill undefined neighborhood    
for i in range(len(content2["Neighbourhood"])):
    if content2['Neighbourhood'][i] == "Not assigned":
        content2['Neighbourhood'][i] = content2['Borough'][i] 

## Merge Neighbourhood data upon Postcode

In [258]:
# create content2
content3 = content2.copy()
final = content2.copy()
final =final.drop(final.index, inplace=False)

#Merge neighbor data
x = len(content3["Postcode"])-1 #set loop length - 1
temp = content3["Neighbourhood"][0] # Initialize temp

for i in range(x):
#     print("when i equals " + str(i) +" "+str(i != len(content3["Postcode"])))
    if (content3["Postcode"][i+1] == content3["Postcode"][i]):
#         print("temp in IF = "+temp)
        temp = temp+','+content3["Neighbourhood"][i+1] #merge neighbourhood info while postcodes are same
    else :
#         print("temp in ELSE = "+temp)
        series = pd.Series({"Postcode":content3["Postcode"][i],"Borough":content3["Borough"][i],"Neighbourhood":temp}) #write to new Series when the next line is different
        final = final.append(series,ignore_index=True)
        temp=content3["Neighbourhood"][i+1] 

if content3["Postcode"][x] == content3["Postcode"][x-1]: #check if the last one is the same as previous ones
    temp = temp+','+content3["Neighbourhood"][x]
    series = pd.Series({"Postcode":content3["Postcode"][x-1],"Borough":content3["Borough"][x-1],"Neighbourhood":temp})
    final = final.append(series,ignore_index=True)
else :
    series = pd.Series({"Postcode":content3["Postcode"][x],"Borough":content3["Borough"][x],"Neighbourhood":content3["Neighbourhood"][x]})
    final = final.append(series,ignore_index=True)
    


## Shape of pre-processed data

In [237]:
final.shape #shape of pre-processed data

(103, 3)